In [1]:
# This line will make sure that results are replicable

from numpy.random import seed
seed(42)
from tensorflow import set_random_seed
set_random_seed(42)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

%matplotlib inline

import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
import tensorflow as tf

print("libraries all imported, ready to go")

libraries all imported, ready to go


In [3]:
filename = 'processed_data.pkl'
feature = 'class_1'

In [4]:
pickle_in = open(filename,"rb")
df = pickle.load(pickle_in)

In [ ]:
df = df.drop('CompanyNumber', axis=1)

In [ ]:
df.head()

,MortgagesNumMortCharges,MortgagesNumMortOutstanding,MortgagesNumMortPartSatisfied,namechanged,namechanged2,class_1,dAccountsOutstandingTime,dReturnsOutstandingTime,OperatingTime,SIC1_1.0,...,eAccountsAccountCategory_6,eAccountsAccountCategory_7,eAccountsAccountCategory_8,eAccountsAccountCategory_9,eAccountsAccountCategory_10,eAccountsAccountCategory_11,eAccountsAccountCategory_12,eAccountsAccountCategory_13,eAccountsAccountCategory_14,eAccountsAccountCategory_15
0,0,0,0,0,0,False,0.0,1.467655,6.303720,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,False,0.0,0.000000,1.301370,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,False,0.0,0.482192,6.317382,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,False,0.0,1.505906,4.583562,0,...,0,0,0,1,0,0,0,0,0,0
4,1,1,0,1,0,False,0.0,0.591781,7.668493,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
y = df[feature].values
y.shape

(3584436,)

In [ ]:
X = df.drop(feature, axis=1).values
X.shape

(3584436, 114)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

In [ ]:
# Describe the architecture of the model, except for the input layer.
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(20, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
])


# set some additional settings
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    epochs=2, 
                    batch_size=512,
                    verbose=0)

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print('Loss: {:0.3f}, Accuracy: {:0.3f}'.format(loss, acc) )

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
roc_auc = metrics.roc_auc_score(y_test, y_pred)

In [ ]:
plt.figure()
plt.plot(fpr, tpr, color='red', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()